In [4]:
from transformers import AutoTokenizer
import coremltools as ct
import os
import numpy as np


model_path = "coreml_3B_INT4.mlpackage"

/opt/miniconda3/envs/google_gemma2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
TensorFlow version 2.17.0 has not been tested with coremltools. You may run into unexpected errors. TensorFlow 2.12.0 is the most recent version that has been tested.
Torch version 2.4.1 has not been tested with coremltools. You may run into unexpected errors. Torch 2.4.0 is the most recent version that has been tested.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
        "meta-llama/Llama-3.2-3B-Instruct", token=""
    )

mlmodel_fp16 = ct.models.MLModel(model_path)

In [ ]:
inputs = tokenizer("hello world!", return_tensors='np')

tok = inputs['input_ids']

In [ ]:
st_len = tok.shape[-1]

state = mlmodel_fp16.make_state()  # 루프 내에서 상태 초기화

for i in range(10):

    mask = np.full((1, st_len:=st_len+1), -1e9)
    mask = np.triu(mask, k=1)
    mask = np.hstack(
        [np.zeros((1, 1)), mask]
    )[None, None, :, :]

    
    input_dict = {
        'input_ids': tok.astype(np.int32),
        'causal_mask': mask.astype(np.int32)

    }

    preds = mlmodel_fp16.predict(input_dict, state=state)
    
    logits = preds['logits']
    pre_toks = np.argmax(logits[0], axis=-1)

    tok = np.concatenate([tok, [[pre_toks]]], axis= 1)

print(tok)

[[128000  15339   1917      0    320  16225   4488    596     83     11
     719    358    617   1027   7633    439]]
